In [ ]:
import astropy.units as u
import numpy as np
import os
from astropy.table import Table
import copy
os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
import cmasher as cmr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False
%matplotlib inline

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

In [ ]:
id = 13892
galaxies = ResolvedGalaxies([ResolvedGalaxy.init_from_h5(f'/nvme/scratch/work/tharvey/EXPANSE/galaxies/JOF_psfmatched_{id}.h5')])

In [ ]:
galaxies[0].plot_bagpipes_results(run_name='CNST_SFH_RESOLVED_PBP', binmap_type='pixel_by_pixel');

galaxies[0].plot_bagpipes_results(run_name='CNST_SFH_RESOLVED', binmap_type='pixedfit');


In [ ]:




fig, ax = galaxies[0].measure_cog_of_property_map(run_name=run_name,nradii=25, maxrad=20, minrad=0.0,
                                        profile_type='median', replace_nan=False, output_radial_units='reff', density_map=True,
                                        param = param, morphology_run_name = 'F444W_sersic_star_stack_sample',
                                        center='com', binmap_type = binmap_type)
ax[0].set_title(f'{param} Density')       
                                        
#ax.set_yscale('log')


In [ ]:
galaxies[0].get_number_of_bins('voronoi')

galaxies

In [ ]:
param = 'mass_weighted_age' #'beta_[1250,3000]AA' # 'beta_C94' # beta_[1250,3000]AA
run_name =  'CNST_SFH_RESOLVED_VORONOI' #'CNST_SFH_RESOLVED_NOMIN' galfind 
binmap_type = 'voronoi'

redshift_bins = [(4.5, 5.5), (5.5, 6.5), (6.5, 7.5), (7.5, 8.5), (8.5, 10.5)]
redshift_bins = [(4.5, 6), (6, 8), (8, 10.5)]

mass_bins = [(6, 8) ,(9, 11)]

mdata_x = {}
mdata_y = {}
merror_y = {}
'''
failed = 0
for galaxy in galaxies:
    try:
        d = galaxy.radial_scatter_of_property_map(run_name=run_name, param = param, output_radial_units='kpc',
                                        center='com', binmap_type = binmap_type, plot=False, return_values=True, com_run_name="CNST_SFH_RESOLVED_VORONOI")

        mass = galaxy.get_total_resolved_property(run_name=run_name, property='stellar_mass')[1]
    
    except Exception as e:
        print(e)
        failed += 1
        continue


    distances, values, errors = d
    
    redshift = galaxy.redshift
   
    for zbin in redshift_bins:
        if redshift > zbin[0] and redshift < zbin[1]:
            if zbin not in data_x:
                data_x[zbin] = []
                data_y[zbin] = []
                error_y[zbin] = []
            data_x[zbin].extend(distances)
            data_y[zbin].extend(values)
            error_y[zbin].extend(errors)
    
    for mbins in mass_bins:
        if mass > mbins[0] and mass < mbins[1]:
            if mbins not in data_x:
                mdata_x[mbins] = []
                mdata_y[mbins] = []
                merror_y[mbins] = []
            mdata_x[mbins].extend(distances)
            mdata_y[mbins].extend(values)
            merror_y[mbins].extend(errors)
'''


fig, ax = plt.subplots(1, len(redshift_bins), figsize=(15, 5), sharey=True, dpi=150)

for i, zbin in enumerate(redshift_bins):
    x = np.array(data_x[zbin])
    y = np.array(data_y[zbin])
    #error = np.array(error_y[zbin])
    x = x[np.isfinite(y)]
    #error = error[y > -5]
    y = y[np.isfinite(y)]   
    bins=np.linspace(np.min(x), np.max(x), 15)

    nbins=len(bins)
    delta = (bins[1]-bins[0])*0.5

    idx = np.digitize(x, bins)
    median = [np.nanmedian(y[idx==k]) for k in range(nbins)]
    # replace nans with 0
    #median = np.nan_to_num(median)
    ax[i].plot(bins-delta, median, alpha=0.5, lw=1.5, zorder=4, path_effects=[pe.withStroke(linewidth=3, foreground='white')])
    
    npoints = len(x)


    ax[i].scatter(x, y, s=6, alpha=0.8)
    #ax[i].errorbar(x, y, yerr=error, markersize=3, alpha=0.2, lw=2, marker='none', linestyle='none')
    ax[i].text(0.05, 0.95, f'{npoints} bins', transform=ax[i].transAxes, ha='left', va='top')
    ax[i].set_title(f'{zbin[0]} < z < {zbin[1]}')
    fig.supxlabel('Distance from CoM (kpc)')
    
    #bin the data into a 2D histogram
    H, xedges, yedges = np.histogram2d(x, y, bins=(50, 50))

    ax.set_xscale('log')
    #Create a contour plot
    #ax[i].contourf(H.T, extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]), cmap=cmr.infinity, levels=50)
    
    ax[i].set_ylabel(param)
    
    
    #ax[i].set_ylim(-4, 0)

ax[0].set_ylabel(param)

print(f'{failed} galaxies failed to plot')

fig, ax = plt.subplots(1, len(mass_bins), figsize=(15, 5), sharey=True, dpi=150)

for i, mass_bin in enumerate(mass_bins):

    x = np.array(mdata_x[mass_bin])
    y = np.array(mdata_y[mass_bin])
    #error = np.array(merror_y[mass_bin])
    x = x[np.isfinite(y)]

    y = y[np.isfinite(y)]
    bins=np.linspace(np.min(x), np.max(x), 15)


    nbins=len(bins)
    delta = (bins[1]-bins[0])*0.5

    idx = np.digitize(x, bins)
    median = [np.nanmedian(y[idx==k]) for k in range(nbins)]
    # replace nans with 0
    #median = np.nan_to_num(median)
    ax[i].plot(bins-delta, median, alpha=0.5, lw=1.5, zorder=4, path_effects=[pe.withStroke(linewidth=3, foreground='white')])
    
    npoints = len(x)


    ax[i].scatter(x, y, s=6, alpha=0.8)
    #ax[i].errorbar(x, y, yerr=error, markersize=3, alpha=0.2, lw=2, marker='none', linestyle='none')
    ax[i].text(0.05, 0.95, f'{npoints} bins', transform=ax[i].transAxes, ha='left', va='top')
    ax[i].set_title(f'{mass_bin[0]} < log(M) < {mass_bin[1]}')
    fig.supxlabel('Distance from CoM (kpc)')
    
    #bin the data into a 2D histogram
    H, xedges, yedges = np.histogram2d(x, y, bins=(50, 50))

    #Create a contour plot
    #ax[i].contourf(H.T, extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]), cmap=cmr.infinity, levels=50)
    
    ax[i].set_ylabel(param)
    ax[i].set_xscale('log')







In [ ]:
import numpy as np

def calculate_radial_profile(map_2d, center, bin_size=1.0, max_radius=None, statistic='mean'):
    """
    Calculate the radial profile of a 2D map.
    
    Parameters:
    -----------
    map_2d : 2D numpy array
        The input 2D map/image
    center : tuple of (x, y)
        The center coordinates for the radial calculation
    bin_size : float
        The width of each radial bin
    max_radius : float or None
        Maximum radius to calculate. If None, will use the maximum possible radius
    statistic : str
        The statistic to calculate in each radial bin ('mean' or 'median')
    
    Returns:
    --------
    radii : numpy array
        The radial distances (center of each bin)
    profile : numpy array
        The calculated profile values
    std_profile : numpy array
        The standard deviation in each radial bin
    """
    # Create coordinate grids
    y, x = np.indices(map_2d.shape)
    
    # Calculate radial distance for each pixel
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    
    # Determine maximum radius if not specified
    if max_radius is None:
        max_radius = np.max(r)
    
    # Create radial bins
    rbins = np.arange(0, max_radius + bin_size, bin_size)
    
    # Initialize arrays for results
    profile = np.zeros(len(rbins)-1)
    std_profile = np.zeros(len(rbins)-1)
    radii = (rbins[1:] + rbins[:-1])/2
    
    # Calculate statistics for each radial bin
    for i in range(len(rbins)-1):
        # Create mask for current annulus
        mask = (r >= rbins[i]) & (r < rbins[i+1])
        
        if np.any(mask):
            values = map_2d[mask]
            
            if statistic == 'mean':
                profile[i] = np.mean(values)
            elif statistic == 'median':
                profile[i] = np.median(values)
            else:
                raise ValueError("Statistic must be either 'mean' or 'median'")
                
            std_profile[i] = np.std(values)
        else:
            profile[i] = np.nan
            std_profile[i] = np.nan
    
    return radii, profile, std_profile

# Example usage
if __name__ == "__main__":
    # Create a sample 2D Gaussian
    size = 100
    x, y = np.indices((size, size))
    center = (size//2, size//2)
    
    # Create a test image with a Gaussian profile
    sigma = 10
    test_map = np.exp(-((x-center[0])**2 + (y-center[1])**2)/(2*sigma**2))
    
    # Calculate radial profile
    radii, profile, std = calculate_radial_profile(
        test_map,
        center=center,
        bin_size=1.0,
        statistic='mean'
    )
    
    # Optional: Plot the results
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(12, 5))
        
        # Plot original map
        plt.subplot(121)
        plt.imshow(test_map)
        for i in range(len(radii)):
            plt.gca().add_artist(plt.Circle(center, radii[i], fill=False, edgecolor='red', alpha=0.5))
        plt.colorbar(label='Value')
        plt.title('Original 2D Map')
        
        # Plot radial profile
        plt.subplot(122)
        plt.errorbar(radii, profile, yerr=std, fmt='o-')
        plt.xlabel('Radius')
        plt.ylabel('Average Value')
        plt.title('Radial Profile')
        
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("Matplotlib not available for plotting")

In [ ]:
print(galaxies[0].photometry_properties[binmap_type][param].T[0])

In [ ]:
run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes/'
run_name = 'CNST_SFH_RESOLVED_VORONOI'
for galaxy in galaxies:
    try:
        galaxy.load_bagpipes_results(run_name, meta={"binmap_type": "voronoi"}, run_dir = run_dir)
        properties_to_load=["stellar_mass", "sfr", "sfr_10myr"] 
        galaxy.get_resolved_bagpipes_sed(run_name, force=True, run_dir = run_dir)
        # Save the resolved SFH
        galaxy.plot_bagpipes_sfh(
            run_name,
            bins_to_show=["RESOLVED"],
            plot=False,
            force=True,
            run_dir=run_dir,
            overwrite=True,
        )

        # Save the resolved properties
        for prop in properties_to_load:
            galaxy.get_total_resolved_property(
                run_name,
                property=prop,
                log=prop == "stellar_mass",
                force=True,
                correct_map_band='F444W',
                correct_map=['detection', 'SNR_2_all_wide_nobreak'],
                run_dir=run_dir,
            )
    except Exception as e:
        print(e)
        pass